In [ ]:
# install the latest version of the openai library
!pip install openai -q --upgrade

# imports
from openai import OpenAI
import json
import time
import requests

In [ ]:
# Load the OpenAI key from a file
with open('../secrets/openAI_key', 'r') as file:
    openAI_key = file.read().replace('\n', '')

# load the github token
with open('../secrets/github_token', 'r') as file:
    github_token = file.read().replace('\n', '')

In [ ]:
# initalize the client
client = OpenAI(
    api_key = openAI_key
)

In [ ]:
"""
ASK THE CODE API functions
"""

def get_repo_structure(url, branch=None, relativePaths=None):
    # URL of Askthecode API endpoint
    get_repo_structure_url = "https://gabriel.askthecode.ai/api/repository/structure"

    # parameters: url(required), branch(optional), relativePaths(optional)
    params = {
        'url': url,
        'branch': branch,
        'relativePaths': relativePaths
    }

    # headers
    headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {github_token}",  # Ensure your token is correctly referenced
    "Content-Type": "application/json"
    }

    # Make the post request
    response = requests.post(get_repo_structure_url, json=params, headers=headers)

    # check if the response is successful
    if response.status_code == 200:
        # Parsing the response JSON
        structure_response = response.json()
        # Return the response data instead of printing
        return structure_response
    else:
        # Return a formatted error message
        return f"Failed to get the repository structure: {response.status_code}, Reason: {response.reason}"


# get repo content
def get_repo_content(url, filePaths, branch=None, relativePath=None):
    # URL of Askthecode API endpoint
    get_repo_content_url = "https://gabriel.askthecode.ai/api/repository/content"

    # parameters: url(required), filePaths(required), branch(optional), relativePath(optional)
    params = {
        'url': url,
        'filePaths': filePaths,
        'branch': branch,
        'relativePath': relativePath
    }

    # headers
    headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {github_token}", 
    "Content-Type": "application/json"
    }

    # Make the post request
    response = requests.post(get_repo_content_url, json=params, headers=headers)

    # check if the response is successful
    if response.status_code == 200:
        # Parsing the response JSON
        content_response = response.json()
        # Return the response data instead of printing
        return content_response
    
    else:
       return f"Failed to get the repository content: {response.status_code}"

# get repo branches
def get_repo_branches(url):
    # URL of Askthecode API endpoint
    get_repo_branches_url = "https://gabriel.askthecode.ai/api/repository/branch/list"

    # parameters: url(required)
    params = {
        'url': url
    }

   # headers
    headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {github_token}", 
    "Content-Type": "application/json"
    }

    # Make the post request
    response = requests.post(get_repo_branches_url, json=params, headers=headers)

    # check if the response is successful
    if response.status_code == 200:
        # Parsing the response JSON
        branches_response = response.json()
        # Return the response data instead of printing
        return branches_response
    else:
        return f"Failed to get the repository branches: {response.status_code}"
    
# Get commit history
def get_commit_history(url, branch=None, filePath=None):
    # URL of Askthecode API endpoint
    get_commit_history_url = "https://gabriel.askthecode.ai/api/repository/commit/history"

    # parameters: url(required), branch(optional), filePath(optional)
    params = {
        'url': url,
        'branch': branch,
        'filePath': filePath
    }

    # headers
    headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {github_token}", 
    "Content-Type": "application/json"
    }

    # Make the post request
    response = requests.post(get_commit_history_url, json=params, headers=headers)

    # check if the response is successful
    if response.status_code == 200:
        # Parsing the response JSON
        commit_history_response = response.json()
        # Return the response data instead of printing
        return commit_history_response
    else:
        return f"Failed to get the commit history: {response.status_code}"
    
# search repo code
def search_repo_code(url, searchKeywords, branch=None, relativePath=None, searchHitLinesCount=None, skipMatchesCount=None):
    # URL of Askthecode API endpoint
    search_repo_code_url = "https://gabriel.askthecode.ai/api/search/repository/code"

    # parameters: url(required), searchKeywords(required), branch(optional), relativePath(optional), searchHitLinesCount(optional), skipMatchesCount(optional)
    params = {
        'url': url,
        'searchKeywords': searchKeywords,
        'branch': branch,
        'relativePath': relativePath,
        'searchHitLinesCount': searchHitLinesCount,
        'skipMatchesCount': skipMatchesCount
    }

    # headers
    headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {github_token}", 
    "Content-Type": "application/json"
    }

    # Make the post request
    response = requests.post(search_repo_code_url, json=params, headers=headers)

    # check if the response is successful
    if response.status_code == 200:
        # Parsing the response JSON
        search_code_response = response.json()
        # Return the response data instead of printing
        return search_code_response
    else:
        return f"Failed to search the repository code: {response.status_code}"

# search repo commits
def search_repo_commits(url, searchKeywords):
    # URL of Askthecode API endpoint
    search_repo_commits_url = "https://gabriel.askthecode.ai/api/search/repository/commits"

    # parameters: url(required), searchKeywords(required)
    params = {
        'url': url,
        'searchKeywords': searchKeywords
    }

    # headers
    headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {github_token}",
    "Content-Type": "application/json"
    }

    # Make the post request
    response = requests.post(search_repo_commits_url, json=params, headers=headers)

    # check if the response is successful
    if response.status_code == 200:
        # Parsing the response JSON
        search_commits_response = response.json()
        # Return the response data instead of printing
        return search_commits_response
    else:
        return f"Failed to search the repository commits: {response.status_code}"
    
def find_repos(searchKeywords, language=None):
    # URL of Askthecode API endpoint
    find_repos_url = "https://gabriel.askthecode.ai/api/search/repository"

    # parameters: searchKeywords(required), language(optional)
    params = {
        'searchKeywords': searchKeywords,
        'language': language
    }

    # headers
    headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {github_token}",
    "Content-Type": "application/json"
    }

    # Make the post request
    response = requests.post(find_repos_url, json=params, headers=headers)

    # check if the response is successful
    if response.status_code == 200:
        # Parsing the response JSON
        find_repos_response = response.json()
        # Return the response data instead of printing
        return find_repos_response
    else:
        return f"Failed to find repositories: {response.status_code}"


In [ ]:
""""
Test the functions
"""
# Structure retrieval
test_repo_structure_url = get_repo_structure(url="https://github.com/recommenders-team/recommenders")
print("Here is the structure of the repository:")
print(test_repo_structure_url)

# Content retrieval
test_repo_content_url = get_repo_content(url="https://github.com/recommenders-team/recommenders", filePaths=[".github/.codecov.yml"])
print("Here is the content of the repository:")
print(test_repo_content_url)

# Branches retrieval
test_repo_branches_url = get_repo_branches(url="https://github.com/RecandChat/CodeCompass")
print("Here are the branches of the repository:")
print(test_repo_branches_url)

# Commit history retrieval
test_commit_history_url = get_commit_history(url="https://github.com/recommenders-team/recommenders", filePath=".devcontainer/devcontainer.json")
print("Here is the commit history of the repository:")
print(test_commit_history_url)

# Code search
search_keywords = ["Neural News Recommendation"]
test_search_repo_code_url = search_repo_code(url="https://github.com/recommenders-team/recommenders", searchKeywords=search_keywords)
print("Here is the Code search result:")
print(test_search_repo_code_url)

# Commits search
search_keywords = ["Documentation"]
test_search_repo_commits_url = search_repo_commits(url="https://github.com/recommenders-team/recommenders", searchKeywords=search_keywords)
print("Here is the Commits search result:")
print(test_search_repo_commits_url)

# find repos
search_keywords = ["recommenders"]
test_find_repos_url = find_repos(searchKeywords=search_keywords)
print("Here are the repositories found:")
print(test_find_repos_url)

### Assistant API

In [ ]:
tools = [{
    "type": "function",
    "function": {
        "name": "get_repo_structure",
            "description": "Retrieves the Github repository file structure to analyze it and be able to query only relevant files. If the provided URL contains specific branch and directory information, prioritize using that over querying the entire repository structure.",
            "parameters": {
                "type": "object",
                "properties": {
                    "url": {
                        "minLength": 1,
                        "type": "string",
                        "description": "Full Github repository URL provided by the user. For example: https://github.com/[owner]/[repo]/blob/[branch]/[file-path]#[additional-parameters]. The URL MUST be identical to the one, that was provided by the user, you MUST NEVER alter or truncate it. This is crucial for valid responses. You should NEVER truncate additional-parameters.",
                    },
                    "branch": {
                        "type": "string",
                        "description": "Repository branch. Provide only if user has explicitly specified it or the previous plugin response contains it.",
                        "nullable": True
                    },
                    "relativePaths": {
                        "type": "array",
                        "items": {
                            "type": "string"
                        },
                        "description": "Relative paths to retrieve. USE only paths you are certain that exist, NEVER invent them. If the provided URL contains a specific directory path, extract and use it. Otherwise, this should be a directory path or pattern only. Patterns accept * symbol as 'any substring'",
                        "nullable": True
                    }
            },
            "required": ["url"],
            "additionalProperties": False}
        }
    },
    {
    "type": "function",
    "function": {
        "name": "get_repo_content",
        "description": "Retrieves github repository file contents, possibly filtered by file names. Line numbers can be specified in URL as well. NEVER query this endpoint without previously querying get_repo_structure endpoint and when the next step is set to get_repo_structure.",
        "parameters": {
            "type": "object",
            "properties": {
                "url": {
                    "minLength": 1,
                    "type": "string",
                    "description": "Full Github repository URL provided by the user. For example: https://github.com/[owner]/[repo]/blob/[branch]/[file-path]#[additional-parameters]. The URL MUST be identical to the one, that was provided by the user, you MUST NEVER alter or truncate it. This is crucial for valid responses. You should NEVER truncate additional-parameters."
                },
                "branch": {
                    "type": "string",
                    "description": "Repository branch. Provide only if user has explicitly specified it or the previous assistant response contains it. When requesting file from commit, use commit SHA.",
                    "nullable": True
                },
                "relativePath": {
                    "type": "string",
                    "description": "Relative paths to the directory. Provide only if user has explicitly specified it or the previous plugin response contains it.",
                    "nullable": True
                },
                "filePaths": {
                    "type": "array",
                    "items": {
                        "type": "string"
                    },
                    "description": "Files to query the content of. Order them by relevance descendant. This should NEVER contain the repository branch. First determine the branch if possible, and only then the file paths. Pass only if you are sure about the file path, call get_repo_structure otherwise"
                }
            },
            "required": ["url", "filePaths"],
            "additionalProperties": False
        }
    }
},
{
    "type": "function",
    "function": {
        "name": "get_repo_branches",
        "description": "Retrieves a list of branches from a Github repository given its URL.",
        "parameters": {
            "type": "object",
            "properties": {
                "url": {
                    "minLength": 1,
                    "type": "string",
                    "description": "Full Github repository URL provided by the user. For example: https://github.com/[owner]/[repo]/blob/[branch]/[file-path]#[additional-parameters]. The URL MUST be identical to the one, that was provided by the user, you MUST NEVER alter or truncate it. This is crucial for valid responses. You should NEVER truncate additional-parameters."
                }
            },
            "required": ["url"],
            "additionalProperties": False
        }
    }
},
{
    "type": "function",
    "function": {
        "name": "get_commit_history",
        "description": "Returns the commits history for the specific file in the repository. If the file path is not provided, the history of the entire repository will be returned. If the branch is not provided, the default branch will be used.",
        "parameters": {
            "type": "object",
            "properties": {
                "url": {
                    "minLength": 1,
                    "type": "string",
                    "description": "Full Github repository URL provided by the user. For example: https://github.com/[owner]/[repo]/blob/[branch]/[file-path]#[additional-parameters]. The URL MUST be identical to the one, that was provided by the user, you MUST NEVER alter or truncate it. This is crucial for valid responses. You should NEVER truncate additional-parameters."
                },
                "branch": {
                    "type": "string",
                    "description": "Repository branch. Provide only if user has explicitly specified it or the previous assistant response contains it.",
                    "nullable": True
                },
                "filePath": {
                    "type": "string",
                    "description": "Path to the file to request the commit history for. Use path relative to the root directory of the repository.",
                    "nullable": True
                }
            },
            "required": ["url"],
            "additionalProperties": False
        }
    }
},
{
        "type": "function",
        "function": {
            "name": "search_repo_code",
            "description": "Search code by user specified keywords. Use when user explicitly asked to search for something. Otherwise prefer to fetch the repository structure. Invoke only with user-specified, specific keywords (e.g., file, class, method names). Avoid generic terms.",
            "parameters": {
                "type": "object",
                "properties": {
                    "url": {
                        "minLength": 1,
                        "type": "string",
                        "description": "Full Github repository URL provided by the user. For example: https://github.com/[owner]/[repo]/blob/[branch]/[file-path]#[additional-parameters]. The URL MUST be identical to the one, that was provided by the user, you MUST NEVER alter or truncate it. This is crucial for valid responses. You should NEVER truncate additional-parameters."
                    },
                    "searchKeywords": {
                        "type": "array",
                        "items": {
                            "type": "string"
                        },
                        "description": "Search keywords. Invoke only with user-specified keywords. Never use keywords that are not part of the user prompt. When user asks to search for function definitions in a specific file (not directory) and you cannot parse them from file content, pass function keyword relevant for the file language.."
                    },
                    "branch": {
                        "type": "string",
                        "description": "Repository branch. Provide only if user has explicitly specified it or the previous plugin response contains it. When requesting file from commit, use commit SHA.",
                        "nullable": True
                    },
                    "relativePath": {
                        "type": "string",
                        "description": "Relative path to the file or directory to search in. Provide only if user has explicitly specified it or the previous plugin response contains it.",
                        "nullable": True
                    },
                    "searchHitLinesCount": {
                        "type": "integer",
                        "description": "Number of lines to retrieve. Set only when explicitly asked to retrieve the specified amount of lines by the user.",
                        "format": "int32",
                        "nullable": True
                    },
                    "skipMatchesCount": {
                        "type": "integer",
                        "description": "Number of matches to skip in the file. use only when user is searching over file and you need to search for matches that were omitted from the previous search request",
                        "format": "int32",
                        "nullable": True
                    }
                },
                "required": ["url", "searchKeywords"],
                "additionalProperties": False
            }
        }
    },
{
    "type": "function",
    "function": {
        "name": "search_repo_commits",
        "description": "Search commits by user specified keywords. Use only when user explicitly asked to search for commits and provided search query.",
        "parameters": {
            "type": "object",
            "properties": {
                "url": {
                    "minLength": 1,
                    "type": "string",
                    "description": "Full Github repository URL provided by the user. For example: https://github.com/[owner]/[repo]/blob/[branch]/[file-path]#[additional-parameters]. The URL MUST be identical to the one, that was provided by the user, you MUST NEVER alter or truncate it. This is crucial for valid responses. You should NEVER truncate additional-parameters."
                },
                "searchKeywords": {
                    "type": "array",
                    "items": {
                        "type": "string"
                    },
                    "description": "Search keywords. Invoke only with user-specified keywords. Never use keywords that are not part of the user prompt."
                }
            },
            "required": ["url", "searchKeywords"],
            "additionalProperties": False
        }
    }
},
{
    "type": "function",
    "function": {
        "name": "find_repos",
        "description": "Search repositories by user specified keywords. Use only when user explicitly asked to search for repositories and provided search query.",
        "parameters": {
            "type": "object",
            "properties": {
                "searchKeywords": {
                    "type": "array",
                    "items": {
                        "type": "string"
                    },
                    "description": "Search keywords. Always use a single, specific keyword that best represents the topic. Avoid using multiple keywords for the same topic. OR logic applied, so providing multiple keywords for the same topic will worsen the results. Keywords should be singular, contain single word and clearly defined for precise searches."
                },
                "language": {
                    "type": "string",
                    "description": "Programming language. Use only when explicitly specified by the user.",
                    "nullable": True
                }
            },
            "required": ["searchKeywords"],
            "additionalProperties": False
        }
    }
}
# Add more tools here
]

In [ ]:
# Create Assistant
assistant = client.beta.assistants.create(
    name = "CodeCompass",
    instructions = "You are a helpful assistant that analyzes code from github repositories and files when given a github url. You will answer questions about the structure of a repository, the content of a files, or any other code-related queries.",
    model = "gpt-3.5-turbo-0125",
    tools = tools
)

In [ ]:
# Utility function to create a message and run

def create_message_and_run(assistant,query,thread=None):
  if not thread:
    thread = client.beta.threads.create()

  message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=query
  )
  run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
  )
  return run,thread

# Utility function to get details of function to be called

def get_function_details(run):

  print("\nrun.required_action\n",run.required_action)

  function_name = run.required_action.submit_tool_outputs.tool_calls[0].function.name
  arguments = run.required_action.submit_tool_outputs.tool_calls[0].function.arguments
  function_id = run.required_action.submit_tool_outputs.tool_calls[0].id

  print(f"function_name: {function_name} and arguments: {arguments}")

  return function_name, arguments, function_id

# Utility function to submit the function response

def submit_tool_outputs(run,thread,function_id,function_response):
    run = client.beta.threads.runs.submit_tool_outputs(
    thread_id=thread.id,
    run_id=run.id,
    tool_outputs=[
      {
        "tool_call_id": function_id,
        "output": str(function_response),
      }
    ]
    ) 
    return run

available_functions = {
    "get_repo_structure": get_repo_structure,
    "get_repo_content": get_repo_content,
    "get_repo_branches": get_repo_branches,
    "get_commit_history": get_commit_history,
    "search_repo_code": search_repo_code,
    "search_repo_commits": search_repo_commits,
    "find_repos": find_repos
}

'''''
def get_gpt_response(messages):
    chat_completion = client.chat.completions.create(
        model = "gpt-3.5-turbo-0125",
        messages = messages,
        functions = tools,
        function_call = "auto"
    )
    return chat_completion
'''''

# execute the function

def execute_function_call(function_name,arguments):
    function = available_functions.get(function_name,None)
    if function:
        arguments = json.loads(arguments)
        results = function(**arguments)
    else:
        results = f"Error: function {function_name} does not exist"
    return results

In [34]:
# Initially, create a new thread outside the loop
thread = client.beta.threads.create()

while True:
    # Prompt user for their query within the loop
    user_input = input("Please enter your query or type 'STOP' to exit: ")
    if user_input.lower() == "stop":
        break

    # Create a new message and run using the existing thread
    run, thread = create_message_and_run(assistant=assistant, query=user_input, thread=thread)

    while True:
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)  # retrieve the run status
        print("run status", run.status)

        if run.status == "requires_action":
            function_name, arguments, function_id = get_function_details(run)
            function_response = execute_function_call(function_name, arguments)
            run = submit_tool_outputs(run, thread, function_id, function_response)
            continue

        if run.status == "completed":  # GPT has produced an output
            messages = client.beta.threads.messages.list(thread_id=thread.id)
            latest_message = messages.data[0]
            text = latest_message.content[0].text.value
            print(text)
            # Do not break here to allow for subsequent queries in the same thread
            break

        time.sleep(1)  # Sleep to prevent overwhelming the API with requests

    # The loop then continues, allowing for further queries in the same thread


run status in_progress
run status in_progress
run status completed
I'm a code analysis assistant here to help you with any queries or questions you may have regarding code repositories, files, structures, or any programming-related topics. Just provide me with the necessary information or URLs, and I'll do my best to assist you! What specific task would you like help with today?
